In [1]:
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
import warnings
from tqdm.notebook import tqdm
import tensorflow as tf


warnings.filterwarnings(action='ignore')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
# 감지기 생성, 기본 가중치 이용
detector = MTCNN()

# 주어진 사진에서 하나의 얼굴 추출
def extract_face(path, required_size=(150, 150)): 
    # 파일에서 이미지 불러오기
    image = Image.open(path)
    
    if image.size[0] > 300:
        image = image.resize((300,300))
    else:
        pass
    
    # 배열로 변환
    pixels = asarray(image)
    
    # 이미지에서 얼굴 감지
    results = detector.detect_faces(pixels)
    # print(results)
    # [{'box': [166, -110, 906, 1318], 'confidence': 0.9870470762252808,
    #   'keypoints': {'left_eye': (497, 468), 'right_eye': (896, 454), 'nose': (706, 741), 
    #   'mouth_left': (542, 951), 'mouth_right': (804, 950)}}]
    
    
    # 첫 번째 얼굴에서 경계 상자 추출
    x1, y1, width, height = results[0]['box']
    # 버그 수정
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # 얼굴 추출
    face = pixels[y1:y2, x1:x2]
    
    # 모델 사이즈로 픽셀 재조정
    image = Image.fromarray(face)
    image = image.resize(required_size)
    image.save(new_folder + filename)
    

# 폴더를 플롯으로 구체화하기
folder = '../data/telemoji/test/disgust/' # 기존 데이터 위치
new_folder = '../data/telemoji/MTCNN/test/disgust/'  # 새로 저장할 위치


# 파일 열거
for filename in tqdm(listdir(folder), total=len(listdir(folder)),position=0,leave=True):
    try:
        # 경로
        path = folder + filename

        # 얼굴 추출
        extract_face(path)
        
    except IndexError:
        pass

  0%|          | 0/403 [00:00<?, ?it/s]